In [1]:
from math import sqrt
import numpy as np
from scipy.interpolate import lagrange

In [2]:
class RandPoly:
    """
    Random and zero free coefficient polynomial
    """

    def __init__(self, n, name='', R = None, fzc=False, p = 1337):
        self.name = name
        self.n = n
        self.p = p
        self.fzc = fzc # free zero coefficient
        if R:
            self.R = R
            # assert(len(R) == n)
        else: 
            self.R = [0] * (n+1) 
            for t in range(self.n+1):
                if t == 0 and fzc is True:
                    self.R[t] = (t, 0)
                else:
                    r = random.randint(1,self.p)
                    self.R[t] = (t, r)

    def poly(self, x):
        s = 0
        for (n, r) in self.R:
            s += r * x ** n
        return s

    
    def poly_str(self):
        """
        outputs the underlying polynomial
        """
        s = ""
        first_zero = self.R[0][1] == 0
        if first_zero:
            for (i, r) in self.R:
                if i == 0:
                    continue
                elif i == 1:
                    s +=  f"{r}x" 
                else:
                    s += f"+{r}x^{i}"
        else:
            for (i, r) in self.R:
                if i == 0:
                    s += f"{r}"
                elif i == 1:
                    s +=  f"+{r}x" 
                else:
                    s += f"+{r}x^{i}"
        if self.name:
            return f"{self.name}(x)={s}"
        else:
            return s

    def __str__(self):
        return self.poly_str()

    def __repr__(self):
        return self.poly_str()

    def __add__(self, other):
        n = min(len(self.R), len(other.R))
        m = max(len(self.R), len(other.R))
        R = [0] * m
        for i in range(m):
            if i < n:
                R[i] = (i, self.R[i][1] + other.R[i][1])
            elif i < len(self.R):
                R[i]= (i, self.R[i][1])
            elif i < other.R:
                 R[i][1] = (i, other.R[i][1])
        
        return RandPoly(n=self.n, R=R)
        

In [3]:
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)

In [4]:
def generate_shares(func_array, x_points):
    shares = []
    for func in func_array:
        f = func.poly
        x_point_shares = {}
        for x in x_points:
            x_point_shares[x]=f(x)
        shares.append(x_point_shares)
    return shares

def reconstruct(x_points, shares):
    f = lagrange(x_points, shares)
    return f

In [5]:
def secure_distance_rough(alice_data, server_data, x_points):
     # change for subtraction
     server_data = [x*-1 for x in server_data]

     f = [RandPoly(n=2, R=[(0,alice_data[0]),(1,3),(2,5)]),
          RandPoly(n=2, R=[(0,alice_data[1]),(1,10),(2,12)])]
     g = [RandPoly(n=2, R=[(0,server_data[0]),(1,7),(2,9)]),
          RandPoly(n=2, R=[(0,server_data[1]),(1,13),(2,8)])]
     h = [RandPoly(n=2, R=[(0,0),(1,4),(2,16)]),
         RandPoly(n=2, R=[(0,0),(1,14),(2,6)])]
    
     f_shares = generate_shares(f, x_points)
     g_shares = generate_shares(g, x_points)
     h_shares = generate_shares(h, x_points)

     # server points
     s_1_point2 = f_shares[0][x_points[1]] + g_shares[0][x_points[1]] + h_shares[0][x_points[1]]
     s_1_point3 = f_shares[0][x_points[2]] + g_shares[0][x_points[2]] + h_shares[0][x_points[2]]
     s_2_point2 = f_shares[1][x_points[1]] + g_shares[1][x_points[1]] + h_shares[1][x_points[1]]
     s_2_point3 = f_shares[1][x_points[2]] + g_shares[1][x_points[2]] + h_shares[1][x_points[2]]

     # client points
     s_1_point1 = f_shares[0][x_points[0]] + g_shares[0][x_points[0]] + h_shares[0][x_points[0]]
     s_2_point1 = f_shares[1][x_points[0]] + g_shares[1][x_points[0]] + h_shares[1][x_points[0]]

     # client interpolation
     s_1_points = [s_1_point1, s_1_point2, s_1_point3]
     s_2_points = [s_2_point1, s_2_point2, s_2_point3]

     s_1_func = reconstruct(x_points, s_1_points)
     s_2_func = reconstruct(x_points, s_2_points)

     s0Vals = [s_1_func(0), s_2_func(0)]

     # calc distance
     distance = 0
     for i in range(len(alice_data)):
         distance += s0Vals[i]**2
     distance = sqrt(distance) 

     return distance


In [6]:
def generate_shares_clean(func_array, x_points):
     shares = []
     for func in func_array:
          f = func.poly
          x_point_shares = []
          for x in x_points:
               x_point_shares.append(f(x))
          shares.append(x_point_shares)
     return shares

def calc_share_sums(f_shares, g_shares, h_shares):
     s_x_points = []
     for x_point in range(len(f_shares)):
          s_x = []
          for i, j, k in zip(f_shares[x_point], g_shares[x_point], h_shares[x_point]):
               sum = i+j+k
               s_x.append(sum)
          s_x_points.append(s_x)
     return s_x_points
        

def get_set_of_lagrange_points(s_shares_client, s_shares_server):
     lagrange_set = []
     for feature in range(len(s_shares_client[0])):
          s_x_feature = [s_shares_client[0][feature]]
          for i in range(len(s_shares_server[0])):
               s_x_feature.append(s_shares_server[feature][i])
          lagrange_set.append(s_x_feature)
     return lagrange_set


def secure_distance_clean(alice_data, server_data, x_points):
     # change for subtraction
     server_data = [x*-1 for x in server_data]

     f = [RandPoly(n=2, R=[(0,alice_data[0]),(1,3),(2,5)]),
          RandPoly(n=2, R=[(0,alice_data[1]),(1,10),(2,12)]),
          RandPoly(n=2, R=[(0,alice_data[2]),(1,56),(2,43)]),
          RandPoly(n=2, R=[(0,alice_data[3]),(1,29),(2,34)])]
     
     g = [RandPoly(n=2, R=[(0,server_data[0]),(1,7),(2,9)]),
          RandPoly(n=2, R=[(0,server_data[1]),(1,13),(2,8)]),
          RandPoly(n=2, R=[(0,server_data[2]),(1,42),(2,7)]),
          RandPoly(n=2, R=[(0,server_data[3]),(1,8),(2,18)])]
     
     h = [RandPoly(n=2, R=[(0,0),(1,4),(2,16)]),
         RandPoly(n=2, R=[(0,0),(1,14),(2,6)]),
         RandPoly(n=2, R=[(0,0),(1,42),(2,78)]),
         RandPoly(n=2, R=[(0,0),(1,5),(2,54)])]
    
     f_shares = generate_shares_clean(f, x_points)
     g_shares = generate_shares_clean(g, x_points)
     h_shares = generate_shares_clean(h, x_points)

     f_shares_client = [[x[0] for x in f_shares]]
     f_shares_server = [x[1:] for x in f_shares]

     g_shares_client = [[x[0] for x in g_shares]]
     g_shares_server = [x[1:] for x in g_shares]

     h_shares_client = [[x[0] for x in h_shares]]
     h_shares_server = [x[1:] for x in h_shares]

     s_shares_server = calc_share_sums(f_shares_server, g_shares_server, h_shares_server)
     s_shares_client = calc_share_sums(f_shares_client, g_shares_client, h_shares_client)

     lagrange_points_all_s_x = get_set_of_lagrange_points(s_shares_client, s_shares_server)

     # client interpolation
     s_n_functions = [reconstruct(x_points, shares) for shares in lagrange_points_all_s_x]
     all_s0_vals = [f(0) for f in s_n_functions]


     # calc distance
     distance = 0
     for i in range(len(alice_data)):
         distance += all_s0_vals[i]**2
     distance = sqrt(distance) 

     return distance


In [7]:
f = [RandPoly(n=2, R=[(0,5),(1,3),(2,5)]),
          RandPoly(n=2, R=[(0,7),(1,10),(2,12)])]

generate_shares_clean(f, [1,2,3,4,5])

[[13, 31, 59, 97, 145], [29, 75, 145, 239, 357]]

In [8]:
alice_data = [2,16]
server_data = [49,8]

print("Traditional distance: ", euclidean_distance(alice_data, server_data))

x_points = [2,8,13]
print("Secure distance: ", secure_distance_rough(alice_data, server_data, x_points))

Traditional distance:  47.67598976424087
Secure distance:  47.67598976424068


In [9]:
alice_data = [42,6,6,12]
server_data = [3,458,5464,97]

print("Traditional distance: ", euclidean_distance(alice_data, server_data))

x_points = [2,8,13]
print("Secure distance: ", secure_distance_clean(alice_data, server_data, x_points))

Traditional distance:  5477.482450907533
Secure distance:  5477.482450907534
